<a href="https://colab.research.google.com/github/evgeny-kolonsky/gaza-news/blob/main/WSJ_news_crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wall Street Journal articles counter

In [ ]:
# Eugene Kolonsky 2022-24
# script for news headlines data scrapping from Wall Street Journal web site

In [3]:
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=0f57386d0d9e6e64c199851378a84b67c6185bcc5c4b612677ca07af07a230ea
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=a4d4bd92861bca12255dde92eb99b723a45034d90635e7142818719796ee2898
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1b

In [ ]:
import urllib, urllib.request, time, datetime
from bs4 import BeautifulSoup
from datetime import date, timedelta
from newspaper import Article
import json


site = 'https://www.wsj.com/news/archive/'

# set here start date, final date and filename for data saving


FORMAT = '%Y/%m/%d'
start_day = datetime.datetime.strptime('2023/10/01', FORMAT)
fin_day   = datetime.datetime.strptime('2024/06/13', FORMAT)
filename = 'wsj-news-middleeast.txt'

def get_page(day, page_num):
  url = site + day
  url_page = url + '/?page=' + str(page_num)

  try:
    req = urllib.request.Request(url_page, headers={'User-Agent': 'Mozilla/5.0'})
    resp = urllib.request.urlopen(req, timeout=2)
    respData = resp.read().decode('utf-8')
  except Exception as e:
    respData = ''
  return respData

def get_links(html_page):
  links = {}
  soup = BeautifulSoup(html_page)
  for link in soup.find_all('a'):
      href = str(link.get('href'))
      title = str(link.string)
      if  'world/middle-east' in href and '?mod' not in href:
        #print(href)
        links[href] = title
  return links

def get_day(day):
  links = {}
  for page_num in range(1,10):
    p = get_page(day, page_num)
    more_links = get_links(p)
    if any(link in links.keys() for link in more_links.keys()) or len(more_links)==0:
        break
    else:
        links.update(more_links)
  return links

# clear file
file = open(filename, 'w')
file.close()


day = start_day
while day < fin_day:
  strday = day.strftime(FORMAT)
  urls = get_day(strday)
  lines = ''
  for url in urls:
    title = urls[url]
    publisher = 'WSJ'
    line = f'{day:%Y-%m-%d}\t{publisher}\t{title}\t{url}\n'
    lines += line
    print('.', end='')
    with open(filename, 'a') as f:
      f.write(line)
  day = day + timedelta(days=1)




.........................................................................................................................................................................